# **Single material point simulation - pure shear in elastic material**

In [ ]:
%matplotlib widget
from bmcs_matmod.msx import MSX
import matplotlib.pylab as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore');
from bmcs_matmod.msx.energy_dissipation import EnergyDissipation
from ibvpy.tfunction import TimeFunction, TFSelector, TFCyclicSin, TFCyclicNonsymmetricConstant
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.api import XDomainSinglePoint, MATS3DScalarDamage, TStepBC, BCDof
from ibvpy.tmodel.mats3D import MATS3DElastic
from ibvpy.mathkit.linalg.dense_mtx import DenseMtx

In [ ]:
m_elastic = MATS3DElastic()

In [ ]:
xmodel = XDomainSinglePoint()
m = TStepBC(
    domains=[(xmodel, m_elastic),],
    bc=[BCDof(
            var='u', dof=3, value=-0.01)]
)

m.sim.tline.trait_set(step=0.1)
m.sim.reset()
m.sim.run()

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ((ax1,ax2)) = plt.subplots(1, 2)
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,0])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,1])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,2])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,3])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,4])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,5])

ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,0])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,1])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,2])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,4])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,5])

# Shear foce control

In [ ]:
xmodel = XDomainSinglePoint()
m = TStepBC(
    domains=[(xmodel, m_elastic),],
    bc=[BCDof(
            var='f', dof=3, value=-1)]
)
m.sim.reset()

In [ ]:
m.sim.tline.trait_set(step=0.1)
m.sim.reset()
m.sim.run()

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
fig, ((ax1,ax2)) = plt.subplots(1, 2)
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,0])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,1])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,2])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,3])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,4])
ax1.plot(m.hist.U_t[:,3], m.hist.F_t[:,5])

ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,0])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,1])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,2])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,4])
ax2.plot(m.hist.U_t[:,3], m.hist.U_t[:,5])

# Debugging section

In [ ]:
xmodel = XDomainSinglePoint()
m = TStepBC(
    domains=[(xmodel, m_elastic),],
    bc=[BCDof(
            var='f', dof=3, value=-1)]
)
m.sim.reset()

In [ ]:
self = m.sim.tstep

In [ ]:
self.t_n, self.t_n1 = 0, 1

In [ ]:
self.K.reset_mtx()
f_Eis, K_ks, dof_Es = np.array(
    [s.get_corr_pred(self.U_k, self.t_n, self.t_n1)
     for s in self.fe_domain], dtype=object).T
f_Eis, K_ks, dof_Es

In [ ]:
self.K.sys_mtx_arrays = list(K_ks)  # improve
self.K.n_dofs

In [ ]:
F_ext = np.zeros_like(self.U_k)
self.bcond_mngr.apply(
    self.step_flag, None, self.K, F_ext, self.t_n, self.t_n1
)
F_ext

In [ ]:
F_int = np.bincount(
    np.hstack(np.hstack(dof_Es)),
    weights=np.hstack(np.hstack(f_Eis))
)
F_int

In [ ]:
R = F_ext - F_int
self.K.apply_constraints(R)
R

In [ ]:
dm = DenseMtx(assemb=self.K), R

In [ ]:
'''Perform a single iteration
'''
d_U_k, pos_def = self.K.solve(check_pos_def=True)
if self.debug:
    print('positive definite', pos_def)
self.U_k[:] += d_U_k
self.primary_var_changed = True
self.step_flag = 'corrector'
self.U_k

In [ ]:
du = np.linalg.solve(dm[0].mtx, dm[1])
dm

In [ ]:
np.einsum('...a,a->...', dm[0].mtx, du)

##  end debug